In [55]:
import pandas as pd
import requests
import json
from urllib.parse import urlencode

In [56]:
contact_url = "https://api.hubapi.com/crm/v3/objects/contacts?"
params_dict = {
    "hapikey":"",
    "limit":100
}
contact_lst = []

while True:
    urlparams = urlencode(params_dict)
    get_contact_url = contact_url + urlparams
    response = requests.get(get_contact_url)
    response_dict = json.loads(response.text)
    contact_lst.extend(response_dict["results"])
    if "paging" in response_dict.keys():
        params_dict["after"] = response_dict["paging"]["next"]["after"]
    else:
        break

In [57]:
contacts_dict = {}

for eachD in contact_lst:
    _id = eachD["id"]
    contacts_dict[_id] = {}
    for each_key, each_value in eachD.items():
        if each_key != "id":
            if isinstance(each_value, dict):
                for kk, vv in each_value.items():
                    contacts_dict[_id].update({
                        kk:str(vv)
                    })
            else:
                contacts_dict[_id].update({
                    each_key:str(each_value)
                })

In [58]:
emailst = [value.get("email").strip().lower() for value in contacts_dict.values()
           if value.get("email") != "None"]
NAemailst = [key for key, value in contacts_dict.items()
             if value.get("email") == "None"]

paramslst1 = list(zip(["email" for repetition in range(len(emailst))], emailst))
paramslst2 = list(zip(["vid" for repetition in range(len(NAemailst))], NAemailst))

groupby_100 = len(paramslst1)//100
examine_exception_dict = {}
alldicts = {}

for each_g in range(groupby_100):
    batchlst = paramslst1[each_g*100:each_g*100+100]
    batchparams = urlencode(batchlst)
    get_contact_by_emailst = ("https://api.hubapi.com/contacts/v1/contact/emails/batch/?"
                              + batchparams)
    response = requests.get(get_contact_by_emailst,
                            params={"hapikey":""})
    if response.status_code == 200:
        alldicts.update(json.loads(response.text))
    else:
        examine_exception_dict[str(each_g+1)+"batch"] = "expection"
        
    if each_g == (groupby_100-1):
        batchlst = paramslst1[each_g*100::]
        batchparams = urlencode(batchlst)
        get_contact_by_emailst = ("https://api.hubapi.com/contacts/v1/contact/emails/batch/?"
                                  + batchparams)
        response = requests.get(get_contact_by_emailst,
                                params={"hapikey":""})
        if response.status_code == 200:
            alldicts.update(json.loads(response.text))
        else:
            examine_exception_dict["lastbatch"] = "expection"
            
params = urlencode(paramslst2)
get_contact_by_id = ("https://api.hubapi.com/contacts/v1/contact/vids/batch/?"
                     + params)
response = requests.get(get_contact_by_id,
                        params={"hapikey":""})
if response.status_code == 200:
    alldicts.update(json.loads(response.text))
else:
    examine_exception_dict["NAemailbatch"] = "exception"
    
all_clients_properties_dict = {}

for clientid in alldicts.keys():
    client_properties_dict = alldicts.get(clientid)["properties"]
    for each_property in client_properties_dict.keys():
        client_properties_dict[each_property] = str(
            client_properties_dict[each_property].get("value"))
    all_clients_properties_dict[clientid] = client_properties_dict
    
all_clients_associatedcom_dict = {}

for clientid in alldicts.keys():
    all_clients_associatedcom_dict[clientid] = {}
    each_association = alldicts[clientid].get("associated-company")
    if each_association != None:
        if len(each_association.keys()) > 0:
            for each_key, each_value in each_association.items():
                if isinstance(each_value, dict):
                    for kk, vv in each_value.items():
                        all_clients_associatedcom_dict[clientid].update({
                            kk:str(vv.get("value"))
                        })
                else:
                    all_clients_associatedcom_dict[clientid].update({
                        each_key:str(each_value)
                    })
                    
                    
writer = pd.ExcelWriter("clientsRecords.xlsx", engine="openpyxl")
pd.DataFrame(contacts_dict).T.to_excel(writer, sheet_name="Client Face")
pd.DataFrame(all_clients_properties_dict).T.to_excel(writer, sheet_name="Client Properties Profile")
pd.DataFrame(all_clients_associatedcom_dict).T.to_excel(writer, sheet_name="Client's Associated Company")
writer.save()